## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [2]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [3]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [5]:
from nltk.corpus import stopwords
import re
import nltk
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In [27]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [28]:
# Answer below:
yelp['normalized'] = yelp.text.apply(lambda x: normalize(x))
yelp.head()

,text,sentiment,normalized
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [29]:
from nltk import word_tokenize

words = []
for doc in yelp.normalized:
  for word in word_tokenize(doc):
      words.append(word)

unique = set(words)
nb_unique_words = len(unique)
nb_unique_words

1626

In [30]:
from tensorflow.keras.preprocessing.text import one_hot

vocab_size = nb_unique_words
encoded_text = [one_hot(doc, vocab_size) for doc in yelp.normalized]

In [31]:
encoded_text[0:5]

[[828, 1235, 1265],
 [50, 1095, 186],
 [1095, 1444, 241, 853],
 [918, 96, 260, 644, 1363, 714, 584, 1599, 1235, 1067],
 [84, 362, 503, 598]]

Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

ind_vars = pad_sequences(encoded_text)

In [18]:
ind_vars[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 245,  80, 791], dtype=int32)

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [32]:
from sklearn.model_selection import train_test_split as tts

x = ind_vars
y = np.array(yelp.sentiment)

X_train, X_test, y_train, y_test = tts(x, y, test_size=0.2)

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [33]:
import numpy as np
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

max_words = np.max(ind_vars)
max_words

999

In [37]:
ind_vars.shape[1]

83

In [39]:
X_train.shape[1]

83

In [40]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=X_train.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 83, 100)           162600    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                4256      
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 167,129
Trainable params: 167,129
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [36]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/50
7/7 [==============================] - 2s 89ms/step - loss: 0.6952 - accuracy: 0.5251 - val_loss: 0.6917 - val_accuracy: 0.5255
Epoch 2/50
7/7 [==============================] - 0s 41ms/step - loss: 0.6694 - accuracy: 0.7398 - val_loss: 0.6882 - val_accuracy: 0.5306
Epoch 3/50
7/7 [==============================] - 0s 42ms/step - loss: 0.6330 - accuracy: 0.7523 - val_loss: 0.6801 - val_accuracy: 0.5765
Epoch 4/50
7/7 [==============================] - 0s 37ms/step - loss: 0.5934 - accuracy: 0.8172 - val_loss: 0.6623 - val_accuracy: 0.6224
Epoch 5/50
7/7 [==============================] - 0s 37ms/step - loss: 0.5291 - accuracy: 0.9315 - val_loss: 0.6469 - val_accuracy: 0.5969
Epoch 6/50
7/7 [==============================] - 0s 39ms/step - loss: 0.4673 - accuracy: 0.9362 - val_loss: 0.6505 - val_accuracy: 0.5816
Epoch 7/50
7/7 [==============================] - 1s 84ms/step - loss: 0.4169 - accuracy: 0.8624 - val_loss: 0.6050 - val_accuracy: 0.6582
Epoch 8/50
7/7 [===========